In [22]:
import geopandas as gpd

depositions = gpd.read_file('../data/geopackage-monitor_2025-01-06/natura2000-area-receptor-nitrogen-loads.gpkg', layer='m23').to_crs(epsg=28992)
depositions = depositions[depositions['year']==2020]
depositions['max_deposition'] = depositions.groupby('natura2000_area_code')['deposition'].transform('max')

# read Dutch gemeentes
gemeente_borders = gpd.read_file('../data/gemeente_borders.json')
# convert to common CRS
gemeente_borders = gemeente_borders.to_crs(epsg=28992)

# Calculate centroids for each gemeente
gemeente_borders['centroid_geometry'] = gemeente_borders.geometry.centroid
gemeente_centroids = gpd.GeoDataFrame(gemeente_borders.drop(columns='geometry'),
                                      geometry='centroid_geometry',
                                      crs=gemeente_borders.crs)


# Use sjoin_nearest to find for each centroid the nearest hexagon from depositions.
# We include the distance, deposition and max_deposition columns.
nearest = gpd.sjoin_nearest(gemeente_centroids, 
                            depositions[['geometry', 'deposition', 'max_deposition']],
                            how="left",
                            distance_col="distance")


print(nearest.head())



   id statcode  jrstatcode    statnaam   rubriek  \
0   1   GM0003  2019GM0003  Appingedam  gemeente   
1   2   GM0010  2019GM0010    Delfzijl  gemeente   
2   3   GM0014  2019GM0014   Groningen  gemeente   
3   4   GM0024  2019GM0024   Loppersum  gemeente   
4   5   GM0034  2019GM0034      Almere  gemeente   

               centroid_geometry  index_right  deposition  max_deposition  \
0  POINT (252449.552 593289.368)         9084   14.076298       21.317795   
1   POINT (257826.29 593719.367)         9084   14.076298       21.317795   
2   POINT (237391.098 582033.54)       153988   16.000139       42.990200   
3  POINT (244311.727 595754.623)        40332   13.735153       21.317795   
4   POINT (145428.485 486601.39)       763016   15.785524       31.003220   

       distance  
0  15186.756902  
1   9932.462305  
2   9606.695788  
3  13290.371941  
4  10745.317562  
id                                                             6
statcode                                           